In [1]:
import sys
sys.path.append('../')
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [2]:
import afm

In [3]:

afm_model = afm.AttentionalFactorizationMachineModel([100 for i in range(22)],8,8,[0.1,0.1])
afm_traced = symbolic_trace(afm_model)

In [4]:
afm_traced.print_readable()

class AttentionalFactorizationMachineModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        linear_offsets = self.linear.offsets
        add = x + linear_offsets;  linear_offsets = None
        linear_fc = self.linear.fc(add);  add = None
        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None
        linear_bias = self.linear.bias
        add_1 = sum_1 + linear_bias;  sum_1 = linear_bias = None
        embedding_offsets = self.embedding.offsets
        add_2 = x + embedding_offsets;  x = embedding_offsets = None
        embedding_embedding = self.embedding.embedding(add_2);  add_2 = None
        getitem = embedding_embedding[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4

'class AttentionalFactorizationMachineModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        linear_offsets = self.linear.offsets\n        add = x + linear_offsets;  linear_offsets = None\n        linear_fc = self.linear.fc(add);  add = None\n        sum_1 = torch.sum(linear_fc, dim = 1);  linear_fc = None\n        linear_bias = self.linear.bias\n        add_1 = sum_1 + linear_bias;  sum_1 = linear_bias = None\n        embedding_offsets = self.embedding.offsets\n        add_2 = x + embedding_offsets;  x = embedding_offsets = None\n        embedding_embedding = self.embedding.embedding(add_2);  add_2 = None\n        getitem = embedding_embedding[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [5]:
afm_traced.graph.print_tabular()

opcode         name                 target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [3]:

class PatternClass(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(1, 1)
        self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
        self.embed_output_dim = 176
        self.atten = nn.Linear(1,1)
        self.projection = torch.nn.Linear(1, 1)
        self.fc = torch.nn.Linear(1, 1)
        self.num_fields = 22

    def inner_product(self,x):
        num_fields = self.num_fields
        row, col = list(), list()
        for i in range(num_fields - 1):
            for j in range(i + 1, num_fields):
                row.append(i), col.append(j)
        p, q = x[:, row], x[:, col]
        return p * q

    def forward(self,x):
        x = self.embed(x + self.offsets)
        inner_product = self.inner_product(x)
        # return inner_product
        attn_scores = F.relu(self.atten(inner_product))
        attn_scores = F.softmax(self.projection(attn_scores), dim=1)
        return self.fc(torch.sum(attn_scores * inner_product, dim=1))

In [7]:
pn = symbolic_trace(PatternClass())

In [8]:
pn.graph.print_tabular()

opcode         name        target                                                  args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [9]:
afm_traced.graph.print_tabular()

opcode         name                 target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [4]:
matcher =  SubgraphMatcher(pn.graph, match_output=False, match_placeholder=False,
                            remove_overlapping_matches=True)
_matches = matcher.match(afm_traced.graph)

NameError: name 'pn' is not defined

In [11]:
_matches

[InternalMatch(anchors=[fc], nodes_map={fc: afm_fc, sum_1: sum_2, mul_1: mul_1, softmax: softmax, projection: afm_projection, relu: relu, atten: afm_attention, mul: mul, getitem: getitem, embed: embedding_embedding, add: add_2, x: x, offsets: embedding_offsets, getitem_1: getitem_1}, placeholder_nodes=[x], returning_nodes=[afm_fc], name_node_map={})]

In [ ]:
afm_model = afm.AttentionalFactorizationMachineModel([100 for i in range(22)],8,8,[0.1,0.1])
afm_traced = symbolic_trace(afm_model)

In [9]:
def gen_pattern_replace_and_matcher_for_afm(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(1, 1)
        self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
        self.embed_output_dim = 176
        self.atten = nn.Linear(1,1)
        self.projection = torch.nn.Linear(1, 1)
        self.fc = torch.nn.Linear(1, 1)
        # self.num_fields = 22

    def inner_product(self,x):
      return x[getitem_node_args[0]] * x[getitem_node_args[1]]

    def forward(self,x):
        x = self.embed(x + self.offsets)
        inner_product = self.inner_product(x)
        # return inner_product
        attn_scores = F.relu(self.atten(inner_product))
        attn_scores = F.softmax(self.projection(attn_scores), dim=1)
        return self.fc(torch.sum(attn_scores * inner_product, dim=1))
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)

  
  atten_node = node_map[pattern_env['atten']]
  atten_node_module = utils.get_target_mod(traced,atten_node.target)
  
  projection_node = node_map[pattern_env['projection']]
  projection_node_module = utils.get_target_mod(traced,projection_node.target)
  
  fc_node = node_map[pattern_env['fc']]
  fc_node_module = utils.get_target_mod(traced,fc_node.target)
  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      self.num_fields = num_field
      self.num_prefix = redency_part_slice[1].stop
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redency_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.unredency_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redency_cross_part_total - self.unredency_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]

      self.attention = atten_node_module
      self.projection = projection_node_module
      self.fc = fc_node_module

    def pn(self,reducey_x,unredency_x):
       redency_x_row, reducey_x_col = list(),list()
       unredency_x_row, unredency_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redency_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             unredency_x_row.append(i),unredency_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return (reducey_x[redency_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),\
              unredency_x[:,unredency_x_row] * unredency_x[:,unredency_x_col],\
              reducey_x[mixed_x_row] * unredency_x[:,mixed_x_col]


    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_part_embed = self.embed(redency_part)
      unredency_part_embed = self.embed(unredency_part)
      redency_part_pn, unredency_part_pn, mixed_part_pn = self.pn(redency_part_embed,unredency_part_embed)
      redency_attn_scores = self.projection(F.relu(self.attention(redency_part_pn)))
      unredency_attn_scores = self.projection(F.relu(self.attention(unredency_part_pn)))
      mixed_rest_attn_scores = self.projection(F.relu(self.attention(mixed_part_pn)))      
      redency_attn_scores = redency_attn_scores.repeat([batch,1,1])
      attn_scores  = torch.concat([redency_attn_scores,unredency_attn_scores,mixed_rest_attn_scores],dim = 1)
      attn_scores = F.softmax(attn_scores, dim=1)
      # return unredency_sum
      redency_attn_scores = attn_scores[0,:self.redency_cross_part_total,:]
      unredency_attn_scores = attn_scores[:,self.redency_cross_part_total:self.redency_cross_part_total+self.unredency_cross_part_total,:]
      mixed_rest_attn_scores = attn_scores[:,self.redency_cross_part_total+self.unredency_cross_part_total:,]
      attn_output = torch.sum(redency_attn_scores * redency_part_pn, dim=1) + torch.sum(unredency_attn_scores * unredency_part_pn, dim=1) + torch.sum(mixed_rest_attn_scores * mixed_part_pn, dim=1)
      return self.fc(attn_output)
  return pattern,ReplacementClass(),_match


In [87]:
afm_model = afm.AttentionalFactorizationMachineModel([100 for i in range(22)],8,8,[0.1,0.1])
afm_traced = symbolic_trace(afm_model)

In [90]:
interp = utils.ProfilingInterpreter(afm_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 20.403146743774414 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_function  getitem                        0.00372028            18.2339
call_function  getitem_1                      0.00365186            17.8985
call_function  mul                            0.00241923            11.8572
call_function  add                            0.00229907            11.2682
call_module    afm_attention                  0.00201297             9.86597
call_module    afm_projection                 0.00101638             4.98148
call_function  relu                           0.000965357            4.73141
call_function  mul_1                          0.000807524            3.95784
call_module    linear_fc                      0.000775099            3.79892
call_function  softmax                        0.000536442            2.62921
call_module    embedding_embedding        

In [91]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_afm(afm_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22)

In [92]:
matches = subgraph_rewriter.replace_pattern_with_filters(afm_traced, pattern, replace,[match])

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [94]:
interp = utils.ProfilingInterpreter(afm_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 15.79904556274414 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_function  add                         0.00236869            14.9926
call_function  getitem_7                   0.00122476             7.75209
call_function  softmax                     0.000989437            6.26264
call_module    attention_2                 0.000893593            5.65599
call_function  mul_5                       0.000732422            4.63586
call_function  concat                      0.000711679            4.50457
call_function  getitem_5                   0.000708103            4.48194
call_function  getitem_4                   0.000699282            4.4261
call_module    attention_1                 0.000670433            4.2435
call_module    linear_fc                   0.000602245            3.81191
call_function  mul_2                       0.000491619            3.1117
call_

In [85]:
sample = torch.ones((4096,22), dtype=torch.long)
modify_mod = GraphModule(afm_traced, afm_traced.graph)

torch.onnx.export(modify_mod,sample,f'afm_modify.onnx')

In [74]:
interp = utils.ProfilingInterpreter(afm_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 14.925241470336914 ms
Op type        Op                  Average runtime (s)    Pct total runtime
-------------  ----------------  ---------------------  -------------------
call_function  add                         0.00180006            12.0605
call_function  getitem_7                   0.00124216             8.32255
call_function  mul_5                       0.00124192             8.32095
call_function  softmax                     0.000983238            6.58775
call_module    attention_2                 0.000786066            5.26669
call_function  getitem_4                   0.000716686            4.80184
call_function  getitem_5                   0.00070715             4.73794
call_module    linear_fc                   0.000674725            4.52069
call_module    attention_1                 0.000495195            3.31784
call_function  mul_1                       0.000464201            3.11017
call_function  concat                      0.00043869             2.93925
c

In [10]:
afm_model = afm.AttentionalFactorizationMachineModel([100 for i in range(100)],32,8,[0.1,0.1])
afm_traced = symbolic_trace(afm_model)

In [15]:
interp = utils.ProfilingInterpreter(afm_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 627.7472972869873 ms
total time: 783.5931777954102 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  getitem                         128.221              16.3632
call_function  getitem_1                       121.187              15.4655
call_function  mul                             120.004              15.3146
call_function  mul_1                           116.572              14.8766
call_module    afm_attention                    60.8704              7.76811
call_function  relu                             32.383               4.13262
call_function  sum_2                            19.7294              2.51781
call_module    afm_projection                   10.998               1.40354
call_function  softmax                           8.53848             1.08966
call_module    embedding_embedding               5.90801             0.753964
cal

In [12]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_afm(afm_traced,
                                                                      (0,slice(None,40,None)),(slice(None,None,None),slice(40,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=100)

In [16]:
matches = subgraph_rewriter.replace_pattern_with_filters(afm_traced, pattern, replace,[match])

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [17]:
interp = utils.ProfilingInterpreter(afm_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 449.93114471435547 ms
total time: 581.6290378570557 ms
Op type        Op                  Average runtime (ms)    Pct total runtime
-------------  ----------------  ----------------------  -------------------
call_function  mul_2                         55.0554              9.46572
call_function  getitem_7                     53.9186              9.27027
call_function  mul_5                         52.5537              9.0356
call_function  getitem_4                     41.9025              7.20434
call_function  mul_1                         41.0187              7.05239
call_function  getitem_5                     40.2176              6.91465
call_function  mul_4                         37.3588              6.42313
call_module    attention_2                   28.8889              4.9669
call_module    attention_1                   23.3145              4.00848
call_function  relu_2                        12.7125              2.18567
call_function  sum_4                 

# torchfx

In [99]:
def workload_afm(dim,batch):
  print(f"now gen workload of afm with config: dim: {dim},batch : {batch}")
  afm_model = afm.AttentionalFactorizationMachineModel([100 for i in range(22)],8,8,[0.1,0.1])


  afm_model_traced_ori = symbolic_trace(afm_model)
  
  afm_model_modify = afm.AttentionalFactorizationMachineModel([100 for i in range(22)],8,8,[0.1,0.1])
  afm_model_traced_modify = symbolic_trace(afm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_afm(afm_model_traced_modify,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22,batch=batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(afm_model_traced_modify, pattern, replace,[match])
  return afm_model_traced_ori,afm_model_traced_modify

# dim = 8

In [100]:
ori,modify = workload_afm(8,1024)

now gen workload of afm with config: dim: 8,batch : 1024


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [103]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.01 ms ± 1.18 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [104]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.28 ms ± 848 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [105]:
ori,modify = workload_afm(8,4096)

now gen workload of afm with config: dim: 8,batch : 4096


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [108]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

9.34 ms ± 2.47 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [109]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

8.22 ms ± 2.01 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 32

In [110]:
ori,modify = workload_afm(32,1024)

now gen workload of afm with config: dim: 32,batch : 1024


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [114]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.37 ms ± 1.29 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [116]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.41 ms ± 1.05 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [117]:
ori,modify = workload_afm(32,4096)

now gen workload of afm with config: dim: 32,batch : 4096


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [122]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

9.43 ms ± 2.22 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [123]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

8.4 ms ± 2.14 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 64

In [124]:
ori,modify = workload_afm(64,1024)

now gen workload of afm with config: dim: 64,batch : 1024


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [125]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.04 ms ± 1.35 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [126]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.48 ms ± 1.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [127]:
ori,modify = workload_afm(64,4096)

now gen workload of afm with config: dim: 64,batch : 4096


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [128]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

9.46 ms ± 2.18 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [129]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

8.25 ms ± 1.57 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 128

In [130]:
ori,modify = workload_afm(128,1024)

now gen workload of afm with config: dim: 128,batch : 1024


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [132]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.07 ms ± 1.27 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [134]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.42 ms ± 1.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [135]:
ori,modify = workload_afm(64,4096)

now gen workload of afm with config: dim: 64,batch : 4096


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [136]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

9.36 ms ± 2.55 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [137]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

8.41 ms ± 2.56 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# torchscript

In [142]:
def workload_afm_script(dim,batch):
  print(f"now gen workload of afm with config: dim: {dim},batch : {batch}")
  afm_model = afm.AttentionalFactorizationMachineModel([100 for i in range(22)],8,8,[0.1,0.1])


  afm_model_traced_ori = symbolic_trace(afm_model)
  
  afm_model_modify = afm.AttentionalFactorizationMachineModel([100 for i in range(22)],8,8,[0.1,0.1])
  afm_model_traced_modify = symbolic_trace(afm_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_afm(afm_model_traced_modify,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22,batch=batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(afm_model_traced_modify, pattern, replace,[match])
  ori = torch.jit.script(afm_model_traced_ori)
  modify = torch.jit.script(afm_model_traced_modify)
  ori.eval()
  modify.eval()
  return ori,modify

# dim = 8

In [143]:
ori,modify = workload_afm_script(8,1024)

now gen workload of afm with config: dim: 8,batch : 1024


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [145]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.74 ms ± 1.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [147]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4 ms ± 1.04 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [148]:
ori,modify = workload_afm_script(8,4096)

now gen workload of afm with config: dim: 8,batch : 4096


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [161]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

18.2 ms ± 2.19 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [158]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

7.52 ms ± 1.79 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 32

In [162]:
ori,modify = workload_afm_script(32,1024)

now gen workload of afm with config: dim: 32,batch : 1024


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [171]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.75 ms ± 942 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [170]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.98 ms ± 957 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [172]:
ori,modify = workload_afm_script(32,4096)

now gen workload of afm with config: dim: 32,batch : 4096


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [175]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

8.81 ms ± 1.52 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [176]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

7.66 ms ± 2.01 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 64

In [177]:
ori,modify = workload_afm_script(64,1024)

now gen workload of afm with config: dim: 64,batch : 1024


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [180]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.85 ms ± 1.12 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [181]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.06 ms ± 1.01 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [182]:
ori,modify = workload_afm_script(64,4096)

now gen workload of afm with config: dim: 64,batch : 4096


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [183]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

9.67 ms ± 4.15 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [185]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

7.66 ms ± 2.06 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 128

In [186]:
ori,modify = workload_afm_script(128,1024)

now gen workload of afm with config: dim: 128,batch : 1024


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [189]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.85 ms ± 1.22 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [190]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.07 ms ± 1.14 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [191]:
ori,modify = workload_afm_script(128,4096)

now gen workload of afm with config: dim: 128,batch : 4096


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [194]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

8.48 ms ± 2.12 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [195]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

7.48 ms ± 1.96 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)
